# Multi-Security Comparison

Compare intraday price paths, normalised returns, and statistical properties
across securities from a parallel multi-security run.

## Data generation

```bash
./build/qrsdp_run --seed 42 --days 5 --securities "AAPL:10000,MSFT:15000,GOOG:20000"
```

In [24]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

import qrsdp_reader as reader
import book_replay as replay
import ohlc

In [25]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")

manifest = reader.load_manifest(RUN_DIR)
symbols = reader.manifest_symbols(manifest)
assert len(symbols) >= 2, f"Need a multi-security run, got symbols={symbols}"

COLOURS = ["#1976D2", "#D32F2F", "#388E3C", "#F57C00", "#7B1FA2", "#0097A7"]
sym_colour = {s: COLOURS[i % len(COLOURS)] for i, s in enumerate(symbols)}

print(f"Run: {RUN_DIR.name}")
print(f"Securities: {', '.join(symbols)}")
for sec in manifest["securities"]:
    print(f"  {sec['symbol']:6s}  p0={sec['p0_ticks']}  days={len(sec['sessions'])}")

Run: run_42
Securities: AAPL, MSFT, GOOG
  AAPL    p0=10000  days=5
  MSFT    p0=15000  days=5
  GOOG    p0=20000  days=5


## 1. Intraday Mid-Price Paths

Overlay the raw mid-price for each security on a single day, sharing the
time axis. Since securities trade at different absolute price levels,
each gets its own y-axis row.

In [26]:
# Pick the first day for intraday comparison
DAY_INDEX = 0
target_date = manifest["securities"][0]["sessions"][DAY_INDEX]["date"]

# Load and replay each security
intraday = {}  # symbol -> {ts_s, mid_ticks, spread_ticks, header}
for sec in manifest["securities"]:
    sym = sec["symbol"]
    sess = sec["sessions"][DAY_INDEX]
    fpath = RUN_DIR / sess["file"]
    hdr = reader.read_header(fpath)
    events = reader.read_day(fpath)
    bk = replay.replay_book(
        events,
        p0_ticks=hdr["p0_ticks"],
        levels_per_side=hdr["levels_per_side"],
        initial_spread_ticks=hdr["initial_spread_ticks"],
        initial_depth=hdr["initial_depth"],
    )
    stride = max(1, len(bk["ts_ns"]) // 20000)
    intraday[sym] = {
        "ts_s": bk["ts_ns"][::stride] / 1e9,
        "mid_ticks": bk["mid_ticks"][::stride],
        "spread_ticks": bk["spread_ticks"][::stride],
        "header": hdr,
        "n_events": len(events),
    }
    print(f"  {sym}: {len(events):>10,} events, "
          f"mid range [{bk['mid_ticks'].min():.0f}, {bk['mid_ticks'].max():.0f}]")

  AAPL:  1,689,650 events, mid range [10000, 10355]
  MSFT:  1,685,427 events, mid range [14793, 15240]
  GOOG:  1,687,837 events, mid range [19991, 20330]


In [27]:
fig_intra = make_subplots(
    rows=len(symbols), cols=1, shared_xaxes=True,
    subplot_titles=[f"{s} — mid-price (ticks)" for s in symbols],
    vertical_spacing=0.06,
)

for i, sym in enumerate(symbols):
    d = intraday[sym]
    fig_intra.add_trace(
        go.Scattergl(
            x=d["ts_s"], y=d["mid_ticks"],
            mode="lines", name=sym,
            line=dict(color=sym_colour[sym], width=1),
        ),
        row=i + 1, col=1,
    )
    fig_intra.update_yaxes(title_text="Price (ticks)", row=i + 1, col=1)

fig_intra.update_xaxes(title_text="Time (s)", row=len(symbols), col=1)
fig_intra.update_layout(
    title=f"Intraday Mid-Price — {target_date}",
    height=280 * len(symbols),
    template="plotly_white",
    showlegend=True,
)
fig_intra.show()

## 2. Normalised Price Paths (Rebased to 100)

Rebase each security's mid-price to 100 at session open so paths at
different absolute levels can be compared on a single y-axis.

In [28]:
fig_norm = go.Figure()

for sym in symbols:
    d = intraday[sym]
    p0 = d["mid_ticks"][0]
    normalised = 100.0 * d["mid_ticks"] / p0
    fig_norm.add_trace(go.Scattergl(
        x=d["ts_s"], y=normalised,
        mode="lines", name=sym,
        line=dict(color=sym_colour[sym], width=1.5),
    ))

fig_norm.add_hline(y=100, line_dash="dash", line_color="grey", opacity=0.5)
fig_norm.update_layout(
    title=f"Normalised Mid-Price (base=100) — {target_date}",
    xaxis=dict(title="Time (s)", rangeslider=dict(visible=True)),
    yaxis=dict(title="Normalised price"),
    height=500,
    template="plotly_white",
)
fig_norm.show()

## 3. Multi-Day Normalised Closing Prices

Chain all days and plot each security's cumulative price path,
normalised to 100 on day 1.

In [29]:
fig_multi_close = go.Figure()

for sym in symbols:
    sec = next(s for s in manifest["securities"] if s["symbol"] == sym)
    closes = []
    dates = []
    for sess in sec["sessions"]:
        fpath = RUN_DIR / sess["file"]
        hdr = reader.read_header(fpath)
        events = reader.read_day(fpath)
        bk = replay.replay_book(
            events,
            p0_ticks=hdr["p0_ticks"],
            levels_per_side=hdr["levels_per_side"],
            initial_spread_ticks=hdr["initial_spread_ticks"],
            initial_depth=hdr["initial_depth"],
        )
        closes.append(float(bk["mid_ticks"][-1]))
        dates.append(sess["date"])

    p0 = float(sec["p0_ticks"])
    norm_closes = [100.0 * c / p0 for c in closes]

    fig_multi_close.add_trace(go.Scatter(
        x=dates, y=norm_closes,
        mode="lines+markers", name=sym,
        line=dict(color=sym_colour[sym], width=2),
        marker=dict(size=7),
    ))

fig_multi_close.add_hline(y=100, line_dash="dash", line_color="grey", opacity=0.5)
fig_multi_close.update_layout(
    title="Multi-Day Normalised Closing Price (base=100 at open of day 1)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Normalised price"),
    height=450,
    template="plotly_white",
)
fig_multi_close.show()

## 4. Intraday Return Distributions

Compare 10-second log-return distributions across securities.
Since all securities use the same intensity model parameters,
the return distributions should be similar when normalised.

In [30]:
return_interval_ns = ohlc.RESOLUTIONS["10s"]

fig_ret = make_subplots(
    rows=1, cols=2,
    subplot_titles=["10s Return Distribution", "QQ-Plot vs Normal"],
    column_widths=[0.55, 0.45],
)

ret_stats = []

for sym in symbols:
    d = intraday[sym]
    ts_full = intraday[sym]["ts_s"] * 1e9  # back to ns for ohlc
    bars = ohlc.compute_ohlc(
        ts_full.astype(np.uint64),
        d["mid_ticks"].astype(np.float64),
        return_interval_ns,
    )
    log_ret = np.diff(np.log(bars["close"].values))
    log_ret = log_ret[np.isfinite(log_ret)]

    # Histogram
    fig_ret.add_trace(
        go.Histogram(
            x=log_ret, name=sym, opacity=0.6,
            marker_color=sym_colour[sym],
            nbinsx=80,
        ),
        row=1, col=1,
    )

    # QQ plot
    sorted_ret = np.sort(log_ret)
    n = len(sorted_ret)
    theoretical = stats.norm.ppf((np.arange(1, n + 1) - 0.5) / n)
    fig_ret.add_trace(
        go.Scattergl(
            x=theoretical, y=sorted_ret,
            mode="markers", name=f"{sym} QQ",
            marker=dict(color=sym_colour[sym], size=2, opacity=0.5),
        ),
        row=1, col=2,
    )

    kurt = stats.kurtosis(log_ret)
    skw = stats.skew(log_ret)
    ret_stats.append({
        "Symbol": sym,
        "Mean": f"{log_ret.mean():.6f}",
        "Std": f"{log_ret.std():.6f}",
        "Skewness": f"{skw:.3f}",
        "Excess Kurtosis": f"{kurt:.3f}",
        "N bars": len(log_ret),
    })

# 45-degree reference line on QQ
qq_range = 4
fig_ret.add_trace(
    go.Scatter(
        x=[-qq_range, qq_range], y=[-qq_range * log_ret.std(), qq_range * log_ret.std()],
        mode="lines", line=dict(color="grey", dash="dash"),
        showlegend=False,
    ),
    row=1, col=2,
)

fig_ret.update_xaxes(title_text="Log return", row=1, col=1)
fig_ret.update_yaxes(title_text="Count", row=1, col=1)
fig_ret.update_xaxes(title_text="Theoretical quantiles", row=1, col=2)
fig_ret.update_yaxes(title_text="Sample quantiles", row=1, col=2)
fig_ret.update_layout(
    title=f"10-Second Return Comparison — {target_date}",
    height=450, template="plotly_white",
    barmode="overlay",
)
fig_ret.show()

pd.DataFrame(ret_stats)

,Symbol,Mean,Std,Skewness,Excess Kurtosis,N bars
0,AAPL,0.000008,0.000382,0.113,0.342,2339
1,MSFT,-0.000002,0.000253,-0.117,0.066,2339
2,GOOG,0.000004,0.000196,0.084,0.271,2339


## 5. Spread Comparison

Overlay bid-ask spread across securities for the same trading day.

In [31]:
fig_spread = go.Figure()

for sym in symbols:
    d = intraday[sym]
    fig_spread.add_trace(go.Scattergl(
        x=d["ts_s"], y=d["spread_ticks"],
        mode="lines", name=sym,
        line=dict(color=sym_colour[sym], width=1),
    ))

fig_spread.update_layout(
    title=f"Bid-Ask Spread Comparison — {target_date}",
    xaxis=dict(title="Time (s)", rangeslider=dict(visible=True)),
    yaxis=dict(title="Spread (ticks)"),
    height=400,
    template="plotly_white",
)
fig_spread.show()

## 6. Cross-Security Summary Table

Per-security aggregate statistics across all days in the run.

In [32]:
import os

summary_rows = []
for sec in manifest["securities"]:
    sym = sec["symbol"]
    total_events = 0
    total_bytes = 0
    opens = []
    closes = []

    for sess in sec["sessions"]:
        fpath = RUN_DIR / sess["file"]
        hdr = reader.read_header(fpath)
        events = reader.read_day(fpath)
        total_events += len(events)
        total_bytes += os.path.getsize(fpath)

        bk = replay.replay_book(
            events,
            p0_ticks=hdr["p0_ticks"],
            levels_per_side=hdr["levels_per_side"],
            initial_spread_ticks=hdr["initial_spread_ticks"],
            initial_depth=hdr["initial_depth"],
        )
        opens.append(float(bk["mid_ticks"][0]))
        closes.append(float(bk["mid_ticks"][-1]))

    overall_return = (closes[-1] - opens[0]) / opens[0] * 100
    daily_returns = [(c - o) / o * 100 for o, c in zip(opens, closes)]

    summary_rows.append({
        "Symbol": sym,
        "Days": len(sec["sessions"]),
        "Total Events": f"{total_events:,}",
        "Total Size (MB)": f"{total_bytes / 1024 / 1024:.1f}",
        "Open (day 1)": f"{opens[0]:.0f}",
        "Close (last)": f"{closes[-1]:.0f}",
        "Overall Return": f"{overall_return:+.2f}%",
        "Max Daily Return": f"{max(daily_returns):+.2f}%",
        "Min Daily Return": f"{min(daily_returns):+.2f}%",
    })

summary_df = pd.DataFrame(summary_rows)
summary_df

,Symbol,Days,Total Events,Total Size (MB),Open (day 1),Close (last),Overall Return,Max Daily Return,Min Daily Return
0,AAPL,5,"8,440,594",105.0,10000,8491,-15.09%,+2.15%,-8.43%
1,MSFT,5,"8,435,257",104.9,15000,15010,+0.07%,+2.77%,-1.97%
2,GOOG,5,"8,436,739",104.9,20000,19232,-3.84%,+0.94%,-2.06%
